In [1]:
from ndnumbers.cplx import Cplx
import ndnumbers.cplx as cplx
import jax.numpy as jnp
import jax

CUDA backend failed to initialize: Found CUDA version 12010, but JAX was built against version 12020, which is newer. The copy of CUDA that is installed must be at least as new as the version against which JAX was built. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


In [3]:
a = Cplx(jnp.array([1, 2]))
a

IndexError: Too many indices for array: array has ndim of 1, but was indexed with 2 non-None/Ellipsis indices.

In [4]:
cplx.unit
-1.0 * cplx.i

(-0.0, i-1.0)

In [5]:
-cplx.i

AttributeError: 'Cplx' object has no attribute 'x'

In [ ]:
z = -cplx.i
z.__abs__()

In [ ]:
z.x.shape

In [ ]:
z.x.shape[-1]

In [ ]:
abs(-cplx.i)

In [ ]:
opts = [cplx.null, cplx.unit, -cplx.unit, cplx.i, -cplx.i]
opt_idx = 0
for opt1 in opts:
    for opt2 in opts:
        
        print(f"{opt_idx: 4d}| {opt1.s} * {opt2.s} = {(opt1 * opt2).s}")
        opt_idx += 1


In [ ]:
opt1.__repr__()
opt1.s

In [ ]:
@jax.jit
def x(a,b,c,d):
    return ((a*b+c)*d).norm

# x(cplx.unit,cplx.i,cplx.unit, 2*cplx.i)

In [ ]:
import jax
# jax.grad(x)(cplx.i,cplx.i,3*cplx.unit, 2*cplx.i)


In [ ]:
import jax

In [ ]:
class csimple:
    def __init__(self, x0, x1):
        self.x0 = x0
        self.x1 = x1
        # verify that the shapes are compatible
        if isinstance(x0, jnp.ndarray):
            if x0.shape != x1.shape:
                raise ValueError(f"Shapes of x0 and x1 are not compatible: {x0.shape} and {x1.shape}")
        elif isinstance(x0, float):
            pass
        else:
            pass # must pass as this may be the jax tracing event
            # raise ValueError(f"Type of x0 is not supported: {type(x0)}")
    def __mul__(self, other):
        return csimple(
                self.x0 * other.x0 - self.x1 * other.x1,
                self.x0 * other.x1 + self.x1 * other.x0)        
    def __repr__(self):
        if isinstance(self.x0, jnp.ndarray):
            if self.x0.shape == ():
                return f"({self.x0:+0.1f} + {self.x1:+0.1f}i)"                            
            else:
                # handle dim-1 arrays
                if self.x0.shape[-1] == 1:
                    return f"({self.x0[0]:+0.1f} + {self.x1[0]:+0.1f}i)"                            
                else:
                    return f"cplx?({self.x0.shape})"                                            
        elif isinstance(self.x0, float):
            return f"({self.x0:+0.1f} + {self.x1:+0.1f}i)"
        else:
            return f"cplx(? + ?i)"
    
    def __abs__(self):
        return jnp.linalg.norm(jnp.array([self.x0, self.x1]))
    def __add__(self, other):
        return csimple(self.x0+other.x0, self.x1+other.x1)
    def __sub__(self, other):
        return csimple(self.x0-other.x0, self.x1-other.x1)
    def __truediv__(self, other):
        return csimple(
                (self.x0 * other.x0 + self.x1 * other.x1) / (other.x0**2 + other.x1**2),
                (self.x1 * other.x0 - self.x0 * other.x1) / (other.x0**2 + other.x1**2))
    def __neg__(self):
        return csimple(-self.x0, -self.x1)
    def __pow__(self, other):
        return csimple(
                (self.x0**2 - self.x1**2) * other.x0 + 2 * self.x0 * self.x1 * other.x1,
                2 * self.x0 * self.x1 * other.x0 + (self.x0**2 - self.x1**2) * other.x1)
    def __float__(self):
        return jnp.nan
    
    # indexing and slicing:
    def __getitem__(self, key):
        return csimple(self.x0[key], self.x1[key])    
    
    @property
    def shape(self):
        """Return the shape of the underlying data."""
        """Note: only tests for the shape of x0, as x1 is expected to have the same shape."""
        return self.x0.shape
    
    @property
    def norm(self):
        return jnp.linalg.norm(jnp.array([self.x0, self.x1]).T, axis=-1)
    
    
    def tree_flatten(node):
        children = (node.x0, node.x1)
        aux_data = ()
        return (children, aux_data)
    
    @classmethod
    def tree_unflatten(cls, aux_data, children):
        # print("tree_unflatten:",aux_data, children)
        return cls(children[0], children[1])
    
a = csimple(1.0, 0.0)
b = csimple(0.0, 1.0)
c = csimple(1.0, 1.0)

import jax.tree_util as tu
tu.register_pytree_node_class(csimple)

@jax.jit
def y(a,b,c):
    return ((a*b)+c).x0

y(a,b,a)
jax.jacobian(y)(a,b,c)

In [ ]:
y(a,b,a)

In [ ]:
jax.grad(y)(a,b,a)

In [ ]:
a/b

In [ ]:
c = csimple(jnp.array([0,0,1,1]), jnp.array([0,1,0,1]))
c

In [ ]:
c[0]

In [ ]:
self_x0 * other_x1 + self_x1 * other_x0

In [ ]:
z=c*c

In [ ]:
z.x0

In [ ]:
z.x1

In [ ]:
z

In [ ]:
z.norm

In [ ]:
d=csimple(1.0,1.0)
jax.grad(lambda d: (d*d).x0)(d)


In [ ]:
jnp.array([c.x0, c.x1]).T.shape